In [3]:
import requests
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import ElementNotInteractableException
from os import makedirs, listdir
from os.path import exists
from bs4 import BeautifulSoup
import csv
from time import sleep
from bs4 import element
import re
# import pandas as pd

In [3]:
driver = Chrome()

# Gazeta Wyborcza i Gazeta.pl

In [2]:
keyword = 'duda'

## Get urls

In [229]:
url_dir = 'data/urls/wyborcza_gazeta/'

In [69]:
urls = []

In [ ]:
for i in range(1, 10001):
    txt_html = requests\
        .get("http://szukaj.gazeta.pl/wyszukaj/artykul?&query="+keyword+"&sortMode=SCORE&pageNumber="+str(i))\
        .text
    soup = BeautifulSoup(txt_html)
    
    elements = soup.find_all('section', 'elem')
    if len(elements) != 0:
        for elem in elements:
            a = elem.header.h3.a
            if a:
                urls.append(a['href'])
    else:
        break

In [188]:
urls = list(set(urls))

In [200]:
len(urls)

10250

In [273]:
def make_dir(desired_dir):
    if not exists(desired_dir):
        makedirs(desired_dir)

In [274]:
make_dir(url_dir)

In [242]:
with open(url_dir+keyword, 'a') as f:
    f.write('\n'.join(urls))

In [ ]:
# nohup python -u ./cmd.py > cmd.log &

In [10]:
739/10

73.9

In [3]:
keyword = 'prezydent'

In [19]:
txt_html = requests\
    .get("http://szukaj.gazeta.pl/wyszukaj/artykul?&query="+keyword+"&sortMode=SCORE&pageNumber=74")\
    .text
soup = BeautifulSoup(txt_html)

In [20]:
len(soup.find_all('section', 'elem'))

11

In [21]:
soup.find('article', 'results-index').header

<header>
					356 727 wyników w czasie 0,005 s
				</header>

In [ ]:
for i in range(1, 10001):
    
    
    elements = soup.find_all('section', 'elem')
    if len(elements) != 0:
        for elem in elements:
            a = elem.header.h3.a
            if a:
                urls.append(a['href'])
    else:
        break

## Get articles

In [12]:
driver.get("http://szukaj.gazeta.pl/wyszukaj/artykul?query=duda&dxx=126213")
sleep(5)
try:
    driver\
        .find_element_by_xpath('//*[@id="rodoNotificationWrapper"]/div[2]/div/div[3]/button')\
        .click()
except:
    pass

In [230]:
with open(url_dir+keyword, 'r') as f:
    urls = f.read().split('\n')

In [288]:
def get_data(url):
    if 'wyborcza.pl' in url:
        title = driver.find_element_by_class_name('art-title').text
        lead = driver.find_element_by_class_name('article-lead').text
        try:
            content = driver.find_element_by_id('artykul').text
        except:
            content = driver.find_element_by_class_name('art_content').text
        img_desc =  "-@@@-".join([x.text for x in driver.find_elements_by_class_name('article-image-desc')])
        comments = "-@@@-".join([x.text for x in driver.find_elements_by_class_name('cBody') if x.text != ''])
        return title, lead, content, img_desc, comments
    elif 'gazeta.pl' in url:
        title = driver.find_element_by_id('article_title').text
        lead = driver.find_element_by_id('gazeta_article_lead').text
        content = driver.find_element_by_class_name('art_content').text
        img_desc =  "-@@@-".join([x.text for x in driver.find_elements_by_class_name('desc')])
        comments = "-@@@-".join([x.find_elements_by_tag_name('p')[3].text for x in driver.find_elements_by_class_name('comment-body') if x.text != ''])
        return title, lead, content, img_desc, comments

In [8]:
def get_data(url):
    if 'wyborcza.pl' in url:
        try:
            title = driver.find_element_by_xpath('//*[@id="art-header"]/div[2]/h1').text
        except:
            try:
                title = driver.find_element_by_class_name('art-title').text
            except:
                try:
                    title = driver.find_element_by_xpath('/html/body/div[5]/div[1]/div/header/div[2]').text
                except:
                    try:
                        title = driver.find_element_by_xpath('/html/body/div[4]/div[1]/div/header/div[2]/h1').text
                    except:
                        try:
                            title = driver.find_element_by_xpath('/html/body/main/div[1]/div/header/div[2]/h1').text
                        except:
                            try:
                                title = driver.find_element_by_xpath('/html/body/div[5]/div[1]/div/header/div[2]/h1').text
                            except:
                                try:
                                    title = driver.find_element_by_xpath('/html/body/div[4]/div[1]/div/header/div[2]').text
                                except:
                                    title = None
        try:
            lead = driver.find_element_by_class_name('article-lead').text
        except:
            try:
                lead = driver.find_element_by_xpath('//*[@id="pagetype_art"]/div[4]/div[2]/section/article/section').text
            except:
                lead = None
        try:
            content = driver.find_element_by_id('artykul').text
        except:
            try:
                content = driver.find_element_by_class_name('art_content').text
            except:
                try:
                    content = driver.find_element_by_xpath('//*[@id="artykul"]').text
                except:
                    try:
                        content = driver.find_element_by_xpath('//*[@id="pagetype_art"]/main/div[2]/section/div/article/section[2]').text
                    except:
                        content = driver.find_element_by_xpath('//*[@id="pagetype_art"]/main/div/section/div[1]/article/section[2]').text
        img_desc =  "-@@@-".join([x.text for x in driver.find_elements_by_class_name('article-image-desc')])
        comments = "-@@@-".join([x.text for x in driver.find_elements_by_class_name('cBody') if x.text != ''])
        return [title, lead, content, img_desc, comments]
    
    elif 'gazeta.pl' in url:
        try:
            title = driver.find_element_by_xpath('//*[@id="article_title"]').text
        except:
            title = driver.find_element_by_xpath('//*[@id="holder_101"]').text
        try:
            lead = driver.find_element_by_id('gazeta_article_lead').text
        except:
            try:
                lead = driver.find_element_by_xpath('//*[@id="gazeta_article_lead"]').text
            except:
                lead = None
        try:
            content = driver.find_element_by_id('artykul').text
        except:
            content = driver.find_element_by_class_name('art_content').text
        img_desc =  "-@@@-".join([x.text for x in driver.find_elements_by_class_name('desc')])
        comments = "-@@@-".join([x.find_elements_by_tag_name('p')[3].text for x in driver.find_elements_by_class_name('comment-body') if x.text != ''])
        return [title, lead, content, img_desc, comments]

In [4]:
# prepare urls
url_dir = 'data/wyborcza_gazeta/urls/'
with open('keywords', 'r') as f:
    keywords = f.read().split('\n')

all_urls = []
for collected_url_file in listdir(url_dir):
    if 'ipynb' not in collected_url_file:
        with open(url_dir+collected_url_file, 'r') as f:
            tmp_urls = f.read().split('\n')
        for tmp_url in tmp_urls:
            all_urls.append(tmp_url)
all_urls = list(set(all_urls))
print(len(all_urls))
all_urls = [x for x in all_urls if x != '']
all_urls = [x for x in all_urls if 'gazeta.pl' in x or 'wyborcza.pl' in x]
print(len(all_urls))

952769
712056


In [14]:
driver.get("https://wyborcza.pl/0,0.html")
try:
    driver\
        .find_element_by_xpath('//*[@id="rodoNotificationWrapper"]/div[2]/div/div[3]/button')\
        .click()
except:
    pass
driver.find_element_by_id('wH_login_form').submit()

driver.find_element_by_id('wyborczaEmail').send_keys('slusarczyk1@wp.pl')
driver.find_element_by_id('wyborczaPassword').send_keys('Sraniejebanko1')
driver.find_element_by_xpath('/html/body/section/section[1]/form/div[4]/button').click()
sleep(1)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="wH_login_form"]"}
  (Session info: chrome=79.0.3945.117)


In [16]:
# get article content
bad_urls = []
wyborcza_content = []
gazeta_content = []
i = 102
error_counter = 0
for n, article_url in enumerate(all_urls[112013:]):
    # getpage
    try:
        driver.get(article_url)
        if 'gazeta.pl' in driver.current_url:
            try:
                gazeta_content.append([article_url] + get_data(driver.current_url))
            except:
                sleep(2)
                gazeta_content.append([article_url] + get_data(driver.current_url))
        elif 'wyborcza.pl' in driver.current_url:
            try:
                wyborcza_content.append([article_url] + get_data(driver.current_url))
            except:
                sleep(2)
                wyborcza_content.append([article_url] + get_data(driver.current_url))
    except Exception as e:
        error_counter = error_counter + 1
        print(e)
        print(article_url)
        print('----')
        break
        try:
            driver.close()
        except:
            pass
        try:
            driver = Chrome(options = opt)
        except:
            pass
        bad_urls.append(article_url)
        if error_counter > 1200:
            break


Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="pagetype_art"]/main/div/section/div[1]/article/section[2]"}
  (Session info: chrome=79.0.3945.117)

https://opole.wyborcza.pl/opole/56,35086,19802783,11-pieknych-studentek-kto-zostanie-miss-uniwersytetu-zdjecia.html
----


In [12]:
driver.find_element_by_id('artykul').text

'W mijającym właśnie roku szkolnym zmian w systemie edukacji i programach nauczania nie brakowało. W wyniku reformy oświaty o wcześniejszym obowiązku szkolnym we wrześniu 2015 roku do klas pierwszych poszły zarówno dzieci siedmioletnie, jak i sześcioletnie. W Lublinie naukę w podstawówkach rozpoczęło w półtora rocznika (wiele sześciolatków dostało odroczenia z poradni pedagogiczno-psychologicznych). W 195 klasach pierwszych uczyło się więc ponad cztery tysiące dzieci.'

In [13]:
test = 'https://wyborcza.pl/1,155290,17928296,Duda___zwiazkowcy___katastrofa_gornictwa.html'

In [14]:
driver.get(test)
# get_data(test)

In [247]:
bad_urls = []
wyborcza_content = []
gazeta_content = []

for n, article_url in enumerate(urls):
    if 'wyborcza.pl' in article_url or 'gazeta.plt' in article_url:
        driver.get(article_url)
        try:
            if 'wyborcza.pl' in article_url:
                wyborcza_content.append(get_data(article_url))
            else:
                gazeta_content.append(get_data(article_url))
        except:
            bad_urls.append(article_url)

In [271]:
articles_dir = 'data/articles/wyborcza/'
if not exists(articles_dir):
    makedirs(articles_dir)
    
with open(articles_dir+keyword, 'w') as f:
    writer =  csv.writer(f)
    writer.writerows(wyborcza_content)

In [256]:
articles_dir = 'data/articles/gazeta/'
if not exists(articles_dir):
    makedirs(articles_dir)
    
with open(articles_dir+keyword, 'w') as f:
    writer =  csv.writer(f)
    writer.writerows(gazeta_content)

In [257]:
with open(url_dir+keyword+'_errors', 'a') as f:
    f.write('\n'.join(bad_urls))

### Final scripts

In [26]:
driver = Chrome()

In [3]:
opt = Options()
opt.add_argument("--headless")

In [4]:
driver = Chrome(options = opt)

In [5]:
with open('keywords', 'r') as f:
    keywords = f.read().split('\n')

In [7]:
keywords[:5]

['pis', 'platforma', 'duda', 'morawiecki', 'szydło']

In [8]:
len(keywords)

37

#### Messy one

In [9]:
import requests
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from os import makedirs
from os.path import exists
from bs4 import BeautifulSoup
import csv
from time import sleep

opt = Options()
opt.add_argument("--headless")

with open('keywords', 'r') as f:
    keywords = f.read().split('\n')



def make_dir(desired_dir):
    if not exists(desired_dir):
        makedirs(desired_dir)
        
def get_data(url):
    if 'wyborcza.pl' in url:
        title = driver.find_element_by_class_name('art-title').text
        lead = driver.find_element_by_class_name('article-lead').text
        content = driver.find_element_by_class_name('art_content').text
        img_desc =  "-@@@-".join([x.text for x in driver.find_elements_by_class_name('article-image-desc')])
        comments = "-@@@-".join([x.text for x in driver.find_elements_by_class_name('cBody') if x.text != ''])
        return title, lead, content, img_desc, comments
    elif 'gazeta.pl' in url:
        title = driver.find_element_by_id('article_title').text
        lead = driver.find_element_by_id('gazeta_article_lead').text
        content = driver.find_element_by_class_name('art_content').text
        img_desc =  "-@@@-".join([x.text for x in driver.find_elements_by_class_name('desc')])
        comments = "-@@@-".join([x.find_elements_by_tag_name('p')[3].text for x in driver.find_elements_by_class_name('comment-body') if x.text != ''])
        return title, lead, content, img_desc, comments

    
for keyword in keywords:
    # get urls for that keyword using requests, for both wyborcza.pl and gazeta.pl
    url_dir = 'data/wyborcza_gazeta/urls/'
    urls = []
    
    for i in range(1, 2):
        txt_html = requests\
            .get("http://szukaj.gazeta.pl/wyszukaj/artykul?&query="+keyword+"&sortMode=SCORE&pageNumber="+str(i))\
            .text
        soup = BeautifulSoup(txt_html)

        elements = soup.find_all('section', 'elem')
        if len(elements) != 0:
            for elem in elements:
                a = elem.header.h3.a
                if a:
                    urls.append(a['href'])
        else:
            break
        
    # save scrapped urls
    make_dir(url_dir)
    urls = list(set(urls))
    with open(url_dir+keyword, 'w') as f:
        f.write('\n'.join(urls))
    
    # initialize selenium
    driver = Chrome(options = opt)
    driver.get("https://wyborcza.pl/0,0.html")
    try:
        driver\
            .find_element_by_xpath('//*[@id="rodoNotificationWrapper"]/div[2]/div/div[3]/button')\
            .click()
    except:
        pass
    driver.find_element_by_id('wH_login_form').submit()
    sleep(3)

    driver.find_element_by_id('wyborczaEmail').send_keys('slusarczyk1@wp.pl')
    driver.find_element_by_id('wyborczaPassword').send_keys('Sraniejebanko1')
    sleep(1)
    driver.find_element_by_xpath('/html/body/section/section[1]/form/div[4]/button').click()
    sleep(3)
    
    # read backed up urls 
    with open(url_dir+keyword, 'r') as f:
        urls = f.read().split('\n')
    
    # get article content
    bad_urls = []
    wyborcza_content = []
    gazeta_content = []

    for article_url in urls:
        if 'wyborcza.pl' in article_url or 'gazeta.plt' in article_url:
            driver.get(article_url)
            try:
                if 'wyborcza.pl' in article_url:
                    wyborcza_content.append(get_data(article_url))
                else:
                    gazeta_content.append(get_data(article_url))
            except:
                bad_urls.append(article_url)
    
    # save articles content for separately for wyborcza and gazeta
    articles_dir = 'data/wyborcza/articles/'
    make_dir(articles_dir)

    with open(articles_dir+keyword, 'w') as f:
        writer =  csv.writer(f)
        writer.writerows(wyborcza_content)
        
    articles_dir = 'data/gazeta/articles/'
    make_dir(articles_dir)

    with open(articles_dir+keyword, 'w') as f:
        writer =  csv.writer(f)
        writer.writerows(gazeta_content)
        
    # and save urls script had problem with
    with open(url_dir+keyword+'_errors', 'a') as f:
        f.write('\n'.join(bad_urls))
        
driver.close()

In [25]:
driver.close()

#### For articles

In [ ]:
import requests
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from os import makedirs, listdir
from os.path import exists
from bs4 import BeautifulSoup
import csv
from time import sleep


# imp functions
def make_dir(desired_dir):
    if not exists(desired_dir):
        makedirs(desired_dir)
        
def get_data(url):
    if 'wyborcza.pl' in url:
        try:
            title = driver.find_element_by_xpath('//*[@id="art-header"]/div[2]/h1').text
        except:
            try:
                title = driver.find_element_by_class_name('art-title').text
            except:
                try:
                    title = driver.find_element_by_xpath('/html/body/div[5]/div[1]/div/header/div[2]').text
                except:
                    try:
                        title = driver.find_element_by_xpath('/html/body/div[4]/div[1]/div/header/div[2]/h1').text
                    except:
                        try:
                            title = driver.find_element_by_xpath('/html/body/main/div[1]/div/header/div[2]/h1').text
                        except:
                            try:
                                title = driver.find_element_by_xpath('/html/body/div[5]/div[1]/div/header/div[2]/h1').text
                            except:
                                try:
                                    title = driver.find_element_by_xpath('/html/body/div[4]/div[1]/div/header/div[2]').text
                                except:
                                    title = None
                    
        lead = driver.find_element_by_class_name('article-lead').text
        try:
            content = driver.find_element_by_id('artykul').text
        except:
            content = driver.find_element_by_class_name('art_content').text
        img_desc =  "-@@@-".join([x.text for x in driver.find_elements_by_class_name('article-image-desc')])
        comments = "-@@@-".join([x.text for x in driver.find_elements_by_class_name('cBody') if x.text != ''])
        return [title, lead, content, img_desc, comments]
    elif 'gazeta.pl' in url:
        title = driver.find_element_by_xpath('//*[@id="article_title"]').text
        lead = driver.find_element_by_id('gazeta_article_lead').text
        try:
            content = driver.find_element_by_id('artykul').text
        except:
            content = driver.find_element_by_class_name('art_content').text
        img_desc =  "-@@@-".join([x.text for x in driver.find_elements_by_class_name('desc')])
        comments = "-@@@-".join([x.find_elements_by_tag_name('p')[3].text for x in driver.find_elements_by_class_name('comment-body') if x.text != ''])
        return [title, lead, content, img_desc, comments]


# prepare urls
url_dir = 'data/wyborcza_gazeta/urls/'
with open('keywords', 'r') as f:
    keywords = f.read().split('\n')

all_urls = []
for collected_url_file in listdir(url_dir):
    if 'ipynb' not in collected_url_file:
        with open(url_dir+collected_url_file, 'r') as f:
            tmp_urls = f.read().split('\n')
        for tmp_url in tmp_urls:
            all_urls.append(tmp_url)
all_urls = list(set(all_urls))
print(len(all_urls))
all_urls = [x for x in all_urls if x != '']
all_urls = [x for x in all_urls if 'gazeta.pl' in x or 'wyborcza.pl' in x]
print(len(all_urls))


# initialize selenium
opt = Options()
opt.add_argument("--headless")
driver = Chrome(options = opt)
# driver = Chrome()
driver.get("https://wyborcza.pl/0,0.html")
try:
    driver\
        .find_element_by_xpath('//*[@id="rodoNotificationWrapper"]/div[2]/div/div[3]/button')\
        .click()
except:
    pass
driver.find_element_by_id('wH_login_form').submit()

driver.find_element_by_id('wyborczaEmail').send_keys('slusarczyk1@wp.pl')
driver.find_element_by_id('wyborczaPassword').send_keys('Sraniejebanko1')
driver.find_element_by_xpath('/html/body/section/section[1]/form/div[4]/button').click()
sleep(1)


# get article content
bad_urls = []
wyborcza_content = []
gazeta_content = []
i = 0
for n, article_url in enumerate(all_urls[:300]):
    # getpage
    try:
        driver.get(article_url)
        if 'gazeta.pl' in article_url:
            gazeta_content.append([article_url] + get_data(article_url))
        elif 'wyborcza.pl' in article_url:
            wyborcza_content.append([article_url] + get_data(article_url))
        print(':)')
        print(article_url)
    except Exception as e:
        print(e)
        print(article_url)
        bad_urls.append(article_url)
    # every 100 pages save to new file good and bad ones
    if n % 200 == 0:
        i += 1
        
        articles_dir = 'data/gazeta/articles/'
        make_dir(articles_dir)
        with open(articles_dir + str(i), 'w') as f:
            writer =  csv.writer(f)
            writer.writerows(gazeta_content)
            
        articles_dir = 'data/wyborcza/articles/'
        make_dir(articles_dir)
        with open(articles_dir + str(i), 'w') as f:
            writer =  csv.writer(f)
            writer.writerows(wyborcza_content)
    
with open(url_dir+'errors', 'w') as f:
    f.write('\n'.join(bad_urls))
    
driver.close()

# Fakt

In [ ]:
# get done keywords and ignore them
# get downloaded urls, remove anti-bot captcha 

# for that keyword get url list
# save the list
# filter them (without captcha) using done urls

# for that keyword get articles content
# or save it to errors

# save articles to one file


# fakt
#   urls/<keyword>
#   articles/<keyword>
#   done_urls
#   errors_urls

In [ ]:
# !!! The source of articles is fakt, so the main folder I save thing is fakt, but there are sometimes articles 
# from other newspaper (same holding), onet.pl

In [ ]:
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import ElementNotInteractableException
from os import makedirs, listdir
import csv
from time import sleep



o = Options()
o.add_argument('--headless')
driver = Chrome(options = o)


done_keywords = listdir('data/fakt/urls')
with open('keywords_pl', 'r') as f:
    keywords = f.read().split('\n')
keywords = [x for x in keywords if x not in done_keywords]
    
    
driver.get('https://www.fakt.pl/szukaj?q=duda')
try:
    driver.find_element_by_xpath('//*[@id="Fakt"]/div[3]/div[1]/div[2]/div/div[3]/button[2]').click()
except:
    pass


for keyword in keywords:
    driver.get('https://www.fakt.pl/szukaj?q='+keyword)
    
    err_count = 0
    for i in range(250):
        try:
            driver.find_element_by_class_name('bottomarrow').click()
        except ElementNotInteractableException:
            sleep(1)
            err_count = err_count + 1
            continue
        sleep(3)

        if err_count > 20:
            break
            
    result_elements = driver.find_element_by_class_name('searchResults')\
      .find_elements_by_tag_name('a')
    result_links = [x.get_attribute('href') for x in result_elements if x]
            
    with open('data/fakt/urls/'+keyword, 'a') as f:
        f.write('\n'.join(result_links))
        
    with open('data/fakt/urls/'+keyword, 'r') as f:
        article_urls = list(set(f.read().split('\n')))
    
    not_handled_urls = []
    error_urls = []
    onet_content = []
    fakt_content = []
    
    for article_url in article_urls:
        driver.get(article_url)
        if 'onet.pl' in article_url:
            if len(driver.find_elements_by_xpath('//*[@id="mainPageBody0"]/section/div/article/main/div/div/div/a')) == 1:
                try:
                    driver.find_element_by_xpath('//*[@id="mainPageBody0"]/section/div/article/main/div/div/div/a').click()
                    title = driver.find_element_by_class_name('detailTitle').text.strip()
                    short = driver.find_element_by_class_name('hyphenate').text.strip()
                    long = " ".join([x.text for x in driver.find_elements_by_class_name('hyphenate') if len(x.find_elements_by_tag_name('a')) == 0])
                    desc = "-@@@-".join([x.text for x in driver.find_elements_by_class_name('imgDesc')])
                    comments = None
                    onet_content.append([article_url, title, short, long, desc, comments])
                except:
                    error_urls.append(article_url)
            else:
                not_handled_urls.append(article_url)
        elif 'fakt.pl' in article_url:
            try:
                title = driver.find_element_by_class_name('title').text.strip()
                short = driver.find_element_by_class_name('hyphenate').text.strip()
                long = " ".join([x.text for x in driver.find_elements_by_class_name('hyphenate') if len(x.find_elements_by_tag_name('a')) == 0])
                desc = "-@@@-".join([x.text for x in driver.find_elements_by_class_name('imgDesc')])
                comments = None
                fakt_content.append([article_url, title, short, long, desc, comments])
            except:
                error_urls.append(article_url)
        else:
            not_handled_urls.append(article_url)
            
    with open('data/fakt/articles/'+keyword, 'a') as f:
        writer =  csv.writer(f)
        writer.writerows(fakt_content)
        
with open('data/fakt/error_urls', 'a') as f:
    f.write('\n'.join(error_urls))
with open('data/fakt/not_handled_urls', 'a') as f:
    f.write('\n'.join(not_handled_urls))

In [ ]:
o = Options()
o.add_argument('--headless')
driver = Chrome(options = o)

In [68]:
done_urls = listdir('data/fakt/urls')

In [121]:
o = Options()
o.add_argument('--headless')
driver = Chrome(options = o)

In [128]:
driver = Chrome()

In [129]:
driver.get('https://www.fakt.pl/szukaj?q='+keyword)

In [124]:
try:
    driver.find_element_by_xpath('//*[@id="Fakt"]/div[3]/div[1]/div[2]/div/div[3]/button[2]').click()
except:
    pass

In [125]:
err_count = 0
for i in range(300):
    try:
        driver.find_element_by_class_name('bottomarrow').click()
    except ElementNotInteractableException:
        sleep(1)
        err_count = err_count + 1
        continue
    sleep(3)
    
    if err_count > 20:
        break

In [126]:
result_elements = driver.find_element_by_class_name('searchResults')\
      .find_elements_by_tag_name('a')
result_links = [x.get_attribute('href') for x in result_elements if x]

In [3]:
driver.get('https://www.fakt.pl/pieniadze/zakupy/wlochy-sklep-internetowy-oferuje-karnawalowe-stroje-uchodzcow-fronda-oburzona/ez6ze49#slajd-1')

In [7]:
title = driver.find_element_by_class_name('title').text.strip()
short = driver.find_element_by_class_name('leadDetail').text.strip()
article_body = driver.find_element_by_class_name('articleBody')
long = " ".join([x.text for x in article_body.find_elements_by_class_name('hyphenate') if len(x.text.split(' ')) > 10])
desc = "-@@@-".join([x.text for x in driver.find_elements_by_class_name('imgDesc')])
comments = None

In [8]:
driver.current_url

'https://www.fakt.pl/pieniadze/zakupy/wlochy-sklep-internetowy-oferuje-karnawalowe-stroje-uchodzcow-fronda-oburzona/ez6ze49?utm_source=_viasg_fakt&utm_medium=direct&utm_campaign=leo_automatic&srcc=ucs&utm_v=2'

In [127]:
not_handled_urls = []
error_urls = []
onet_content = []
fakt_content = []

for article_url in result_links:
    driver.get(article_url)
    if 'onet.pl' in article_url:
        if len(driver.find_elements_by_xpath('//*[@id="mainPageBody0"]/section/div/article/main/div/div/div/a')) == 1:
            try:
                driver.find_element_by_xpath('//*[@id="mainPageBody0"]/section/div/article/main/div/div/div/a').click()
                title = driver.find_element_by_class_name('detailTitle').text.strip()
                short = driver.find_element_by_class_name('hyphenate').text.strip()
                long = " ".join([x.text for x in driver.find_elements_by_class_name('hyphenate') if len(x.find_elements_by_tag_name('a')) == 0])
                desc = "-@@@-".join([x.text for x in driver.find_elements_by_class_name('imgDesc')])
                comments = None
                onet_content.append([article_url, title, short, long, desc, comments])
            except:
                error_urls.append(article_url)
        else:
            not_handled_urls.append(article_url)
    elif 'fakt.pl' in article_url:
        try:
            title = driver.find_element_by_class_name('title').text.strip()
            short = driver.find_element_by_class_name('hyphenate').text.strip()
            long = " ".join([x.text for x in driver.find_elements_by_class_name('hyphenate') if len(x.find_elements_by_tag_name('a')) == 0])
            desc = "-@@@-".join([x.text for x in driver.find_elements_by_class_name('imgDesc')])
            comments = None
            fakt_content.append([article_url, title, short, long, desc, comments])
        except:
            error_urls.append(article_url)
    else:
        not_handled_urls.append(article_url)

KeyboardInterrupt: 

In [ ]:
len(fakt_content)

In [ ]:
not_handled_urls

In [ ]:
error_urls

In [115]:
driver.close()

# Onet

In [3]:
with open('keywords_pl', 'r') as f:
    keywords = f.read().split('\n')

In [4]:
onet_keywords = {
    'sąd+najwyższy': 'sad-najwyzszy',
    'imigranci': 'imigracja',
    'ekologia': 'ekologia',
    'emigracja': 'emigracja',
    'pis': 'pis',
    'platforma': 'po',
    'duda': 'andrzej-duda',
    'morawiecki': 'mateusz-morawiecki',
    'szydło': 'beata-szydlo',
    'kaczyński': 'jaroslaw-kaczynski',
    'kwaśniewski': 'aleksander-kwasniewski',
    'lewica': 'lewica',
    'prawica': 'prawica',
    'lgbt': 'lgbt',
    'unia+europejska': 'unia-europejska',
    'rosja': 'rosja',
    'stany': 'stany-zjednoczone',
    'premier': 'premier',
    'prezydent': 'prezydent',
    'opozycja': 'opozycja',
    'rząd': 'rzad',
    'sejm': 'sejm',
    'polska': 'polska',
    'putin': 'putin',
    'trump': 'donald-trump',
    'ukraina': 'ukraina',
    'media': 'media',
    'bank': 'banki',
    'niemcy': 'niemcy',
    'papież': 'papiez',
    'kościół': 'kosciol',
    'korwin': 'korwin',
    'rydzyk': 'tadeusz-rydzyk',
    'feminizm': 'feminizm',
    'leszek+miller': 'leszek-miller',
}

In [5]:
driver = Chrome()
driver.get('https://wiadomosci.onet.pl/')
try:
    driver\
        .find_element_by_class_name('cmp-intro_options')\
        .find_elements_by_tag_name('button')[1].click()
except:
    pass

In [24]:
for keyword in keywords:
    onet_keyword = onet_keywords['sąd+najwyższy']
    driver.get('https://wiadomosci.onet.pl/'+onet_keyword)
    
    
    
    break

In [13]:
# get urls
for i in range(130):
        
    try:
        driver\
            .find_element_by_class_name('articleDetail')\
            .find_elements_by_class_name('more')[-1].click()
        sleep(2)
    except:
        pass
    
    break

In [27]:
driver\
    .find_element_by_class_name('articleDetail')\
    .find_elements_by_class_name('more')[-1].click()

In [18]:
[
    x.find_element_by_class_name('showMore').find_element_by_class_name('more')
    for x in
    driver.find_elements_by_class_name('articleDetail')
]

[<selenium.webdriver.remote.webelement.WebElement (session="bbcc70e9ec02607a4afed1ca9702fc68", element="938e9207-654f-48b6-8e0f-e69780069f10")>]

In [55]:
urls = [
    x.find_elements_by_tag_name('a')[-1].get_attribute('href')
    for x in 
    driver.find_elements_by_class_name('listItem')
]

In [75]:
with open('data/onet/urls/'+keyword, 'w') as f:
    f.write("\n".join(urls))
    
with open('data/onet/urls/'+keyword, 'r') as f:
    urls = f.read().split('\n')

In [ ]:
error_urls = []
onet_content = []
for url in urls:
    driver.get(url)
    try:
        title = driver.find_element_by_class_name('mainTitle').text.strip()
        short = driver.find_element_by_id('lead').text.strip()
        long = " ".join([
            x.text.replace('REKLAMA\n', '')
            for x in 
            driver.find_element_by_id('detail').find_elements_by_class_name('hyphenate')
        ][:-1])
        img = " ".join([
            x.text
            for x in 
            driver.find_elements_by_class_name('imageDescription')
        ])
        com = None
        onet_content.append([
            title, short, long, img, com
        ])
    except:
        error_urls.append(driver.current_url)

In [ ]:
with open('data/onet/articles/'+keyword, 'w') as f:
    writer =  csv.writer(f)
    writer.writerows(onet_content)

In [76]:
with open('data/onet/error_urls', 'w') as f:
    f.write("\n".join(error_urls))

NameError: name 'error_urls' is not defined

# Newsweek

In [23]:
driver.get('https://www.newsweek.pl/')
sleep(1)
try:
    driver.find_element_by_class_name('cmp-intro_acceptAll').click()
except:
    pass

In [24]:
driver.find_element_by_xpath('//*[@id="log-in-out"]').click()
sleep(2)

In [25]:
driver.find_element_by_xpath('//*[@id="f_login"]').send_keys('slusarczyk41@gmail.com')
driver.find_element_by_xpath('//*[@id="f_password"]').send_keys('Starehaslo1')
driver.find_element_by_xpath('//*[@id="loginForm"]/div[2]/ul[2]/li[1]/input').click()
sleep(2)

In [26]:
with open('keywords_pl', 'r') as f:
    keywords = f.read().split('\n')

In [31]:
for keyword in keywords:
    driver.get('https://www.newsweek.pl/szukaj?q='+keyword)
    urls = [
        x.get_attribute('href')
        for x in
        driver\
            .find_element_by_class_name('searchResults')\
            .find_elements_by_tag_name('a')
    ]
    
    with open('data/newsweek/urls/'+keyword, 'w') as f:
        f.write("\n".join(urls))
    with open('data/newsweek/urls/'+keyword, 'r') as f:
        urls = f.read().split('\n')
    
    newsweek_content = []
    error_urls = []
    for url in urls:
        if 'newsweek.pl' in url:
            driver.get(url)
            try:
                newsweek_content.append([
                    driver.find_element_by_class_name('detailTitle').text.strip(),
                    driver.find_element_by_class_name('lead').text.strip(),
                    " ".join([
                        x.text
                        for x in 
                        driver\
                            .find_element_by_class_name('articleDetail')\
                            .find_elements_by_tag_name('p')
                    ])\
                        .replace('REKLAMA', '')\
                        .replace('\n', '').strip(),
                    None,
                    None
                ])
                break
            except:
                error_urls.append(url)
                
    with open('data/newsweek/articles/'+keyword, 'w') as f:
        writer =  csv.writer(f)
        writer.writerows(onet_content)

In [ ]:
with open('data/newsweek/error_urls', 'w') as f:
    f.write("\n".join(error_urls))

# TVN24

In [36]:
driver.get('https://tvn24.pl/szukaj.html?q=duda&r=1&p=1')
try:
    driver.find_element_by_xpath('//*[@id="rodoLayer"]/div/div[3]/a[2]').click()
except:
    pass

In [50]:
with open('keywords_pl', 'r') as f:
    keywords = f.read().split('\n')

In [51]:
tvn_keywords = {
    'sąd+najwyższy': 'najwyższy',
    'imigranci': 'imigranci',
    'ekologia': 'ekologia',
    'emigracja': 'emigracja',
    'pis': 'pis',
    'platforma': 'platforma',
    'duda': 'duda',
    'morawiecki': 'morawiecki',
    'szydło': 'szydło',
    'kaczyński': 'kaczyński',
    'kwaśniewski': 'kwaśniewski',
    'lewica': 'lewica',
    'prawica': 'prawica',
    'lgbt': 'lgbt',
    'unia+europejska': 'europejska',
    'rosja': 'rosja',
    'stany': 'zjednoczone',
    'premier': 'premier',
    'prezydent': 'prezydent',
    'opozycja': 'opozycja',
    'rząd': 'rząd',
    'sejm': 'sejm',
    'polska': 'polska',
    'putin': 'putin',
    'trump': 'trump',
    'ukraina': 'ukraina',
    'media': 'media',
    'bank': 'bank',
    'niemcy': 'niemcy',
    'papież': 'papież',
    'kościół': 'kościół',
    'korwin': 'korwin',
    'rydzyk': 'rydzyk',
    'feminizm': 'feminizm',
    'leszek+miller': 'miller',
}

In [69]:
for keyword in keywords:
    print(keyword)
    tvn_keyword = tvn_keywords[keyword]
    driver.get('https://tvn24.pl/szukaj.html?q='+tvn_keyword+'&r=1&p=1&page=2')
    
    no_pages = int(int(driver.find_element_by_xpath('//*[@id="tvn24"]/div[2]/div/div[3]/div[4]/div[1]/span').text[1:-1])/10)
    if no_pages > 1000:
        no_pages == 1000
    
    all_urls = []
    for i in range(1, no_pages):
        driver.get('https://tvn24.pl/szukaj.html?q='+tvn_keyword+'&r=1&p=1&page='+str(i))
        all_urls.append([
            x.find_element_by_tag_name('a').get_attribute('href')
            for x in 
            driver\
                .find_element_by_class_name('searchResult')\
                .find_elements_by_class_name('news')
        ])
    keyword_urls = [a for x in all_urls for a in x]
    
    with open('data/tvn/urls/'+keyword, 'w') as f:
        f.write("\n".join(keyword_urls))

In [14]:
with open('keywords_pl', 'r') as f:
    keywords = f.read().split('\n')

In [15]:
all_urls = []
for filename in listdir('data/tvn/urls'):
    with open('data/tvn/urls/'+filename, 'r') as f:
        all_urls.append(f.read().split('\n'))

In [17]:
all_urls = set([a for b in all_urls for a in b])

In [86]:
content = []
errors = []
i = 1
for n, url in enumerate(all_urls):
    if 'tvnmeteo.' not in url and 'eurosport.' not in url:
        try:
            r = requests.get(url)
            soup = BeautifulSoup(r.text, features = 'html.parser').find('article')
            title = soup.find('h1').text.strip()
            if soup.find('p', 'lead-text'):
                short = soup.find('p', 'lead-text').text.strip()
                long = " ".join([
                    x.text.strip()
                    for x in
                    soup\
                        .find('div', 'article-story-content__elements')\
                        .find_all('div', 'article-element article-element--paragraph')
                ])
                imgs = "-@@@-".join([
                    x.text.strip()
                    for x in 
                    soup.find_all('div', 'inner-description__title')
                ])
            else:
                short = soup.find('h2').text.strip()
                long = " ".join(" ".join([
                    x.text.replace('\n', '').replace('\xa0', ' ').strip()
                    for x in
                    soup.find_all('p')
                ]).split())
                if long.find('if (\'undefined\' ==') != -1:
                    long = long[:long.find('if (\'undefined\' ==')]
                if soup.find('span', 'photoTitle'):
                    imgs = soup.find('span', 'photoTitle').text.strip()
                else:
                    imgs = None
            com = None
            content.append([title, short, long, imgs, com])
            
        except Exception as e:
            print(url)
            print(traceback.format_exc())
            errors.append(url)
        
    if n % 1000 == 0 and n != 0:
        with open('data/tvn/articles/'+str(i), 'w') as f:
            writer =  csv.writer(f)
            writer.writerows(onet_content)
        i += 1

KeyboardInterrupt: 

In [84]:
url

'https://eurosport.tvn24.pl/archiwum,133/koledzy-wygrywaja-w-giro-d-italia,58060.html'

In [ ]:
with open('data/tvn/error_urls', 'w') as f:
    f.write("\n".join(errors))

# Dziennik

In [ ]:
# https://www.dziennik.pl/szukaj?c=1&b=1&o=1&s=0&search_term=&q=duda&page=41#result

# TVP Info

In [50]:
r = requests.get('https://www.tvp.info/szukaj?query=s%C4%85d+naj&page=150')
soup = BeautifulSoup(r.text)

In [51]:
js = [
    x
    for x in
    soup.find_all('script')
    if '"items"' in str(x)
][0].text

In [52]:
urls = [
    'https://wiadomosci.tvp.pl'+x.replace('https:\\/\\/wiadomosci.tvp.pl', '').replace('\\','')
    for x in
    re.findall(r'"url" : "(.*?)",', js)
]

In [53]:
print(len(urls))

10


In [54]:
u = [[1,2], [3,4]]

In [55]:
[x for y in u for x in y]

[1, 2, 3, 4]

In [4]:
files = []
for filename in listdir('data/tvp/urls/'):
    with open('data/tvp/urls/'+filename, 'r') as f:
        files.append(f.read().split('\n'))

In [5]:
urls = list(set([x for y in files for x in y]))

In [6]:
len(urls)

30251

In [ ]:
articles_list = []
i = 1
for n, url in enumerate(urls):
    try:
        r = requests.get(url.replace('.pl', '.info').replace('wiadomosci.', ''))
        soup = BeautifulSoup(r.text)
        articles_list.append([
            soup.find('h1').text.strip(),
            soup.find('p', 'am-article__heading article__width').text.strip(),
            " ".join([
                x.text.replace('\n', '').replace('\t', '').replace('\r', '').replace('#wieszwiecejPolub nas', '').strip()
                for x in
                soup.find_all('p', 'am-article__text article__width')
            ]),
            None,
            None,
        ])
    except Exception as e:
        print(url, e)
        
    if n % 1000 == 0 and n != 0:
        with open('data/tvp/articles/'+str(i), 'w') as f:
            writer = csv.writer(f)
            writer.writerows(articles_list)
        articles_list = []
        i += 1

# Sputnik

In [25]:
with open('keywords_pl', 'r') as f:
    keywords = f.read().split('\n')

In [30]:
driver.get("https://pl.sputniknews.com/search/?query="+keywords[-1])

In [13]:
from selenium.common.exceptions import NoSuchElementException

In [14]:
for i in range(100):
    try:
        driver\
            .find_element_by_class_name('search')\
            .find_element_by_class_name('m-more')\
            .click()
        sleep(1)
    except NoSuchElementException:
        break
    else:
        pass

In [18]:
urls = list(set([
    x.find_element_by_tag_name('a').get_attribute('href')
    for x in
    driver.find_elements_by_class_name('b-plainlist__title')
]))

In [23]:
print(len(urls))

158


In [39]:
with open('data/sputnik/urls/'+keyword, 'w') as f:
    f.write("\n".join(urls))

In [57]:
with open('data/sputnik/urls/'+keyword, 'r') as f:
    urls = f.read().split('\n')

In [91]:
sputnik_content = []
for url in urls:
    try:
        r = requests.get('https://pl.sputniknews.com/polska/2019122411556887-sad-najwyzszy-ma-zastrzezenia-do-ustawy-o-sadach-sputnik-polska/')
        soup = BeautifulSoup(r.text, features = 'html.parser')
        sputnik_content.append([
            soup.find('h1').text.strip(),
            soup.find('div', 'b-article__lead').text.strip(),
            " ".join(" ".join([
    x.text.strip() if type(x) != element.NavigableString else x.strip()
    for x in
    soup.find('div', 'b-article__text')
]).replace('Sputnik', '').replace('©', '').replace('AFP', '').replace('2019', '').replace('Alexey Vitvitsky', '').replace('Wojtek Radwanski', '').split()),
            None,
            None,
        ])
    except Exception as e:
        print(e)

In [56]:
print(len(sputnik_content))

157


In [ ]:
with open('data/sputnik/articles/'+keyword, 'w') as f:
    writer =  csv.writer(f)
    writer.writerows(sputnik_content)